# Comparação de Algoritmos de Busca em Grafos — IA UFOP

**Problema:** Roteamento em Ouro Preto com função de custo (declividade, rugosidade, congestionamento).

**Algoritmos:** Dijkstra, A* e D* Lite.

**Cenários:** baseline, evento (ruas interditadas), clima (chuva — ladeiras com peso 200%).

In [2]:
import sys, os
sys.path.insert(0, '.')
from src.graph import build_ouro_preto_example, build_op_graph
from src.algorithms import dijkstra, a_star, d_star_lite
from src.scenarios import (
    apply_event_scenario, apply_climate_scenario, apply_climate_scenario_by_region,
    apply_congestion_scenario_by_edge, apply_barriers, reset_scenarios,
    OURO_PRETO_BLOCKED_CASE1, OURO_PRETO_CONGESTED_CASE2, OURO_PRETO_RAIN_REGIONS_CASE3,
    OURO_PRETO_BLOCKED_CASE4, OURO_PRETO_CONGESTED_CASE4, OURO_PRETO_BLOCKED_CASE5,
    OURO_PRETO_CONGESTED_CASE6, OURO_PRETO_BLOCKED_CASE7, OURO_PRETO_RAIN_REGIONS_CASE7,
)
from src.metrics import measure_latency_ms

## 1. Grafo de exemplo (Ouro Preto)

Vértices: Praça Tiradentes, Terminal, Campus UFOP, Rua São José, Diogo de Vasconcelos, Rua do Pilar, Xavier da Veiga, Ladeira da Barra.
Arestas: distância, declividade (%), rugosidade (pé de moleque ~1.57), volume/capacidade.

In [ ]:
api_key = os.getenv('GOOGLE_MAPS_API_KEY')
G = build_op_graph(levels=300, use_cache=True)
print('Vértices:', list(G.nodes()))
print('Arestas (origem -> destino):', list(G.edges()))

build_op_graph: cache de vizinhos carregado (162 ways em cache)
build_op_graph: nivel 1/6 — fronteira=1 ways, vizinhos obtidos=10, novos=9
build_op_graph: nivel 2/6 — fronteira=9 ways, vizinhos obtidos=24, novos=14
build_op_graph: nivel 3/6 — fronteira=14 ways, vizinhos obtidos=55, novos=35
build_op_graph: nivel 4/6 — fronteira=35 ways, vizinhos obtidos=85, novos=41
build_op_graph: nivel 5/6 — fronteira=41 ways, vizinhos obtidos=123, novos=56
build_op_graph: 156 ruas, 460 arestas (levels=6)
Grafo OP (OSM) salvo em /Users/eduardosilva/Documents/IA/comparacao_algoritmos_busca/cache/grafo_op_osm.gpickle
Vértices: ['rua_rio_piracicaba', 'avenida_das_andorinhas', 'estrada_do_mosteiro', 'way_196416349', 'way_385991379', 'rua_rio_verde', 'way_385994083', 'way_385994085', 'way_386020340', 'praça_frei_luiz_maria_sartori', 'rua_vinte_e_quatro_de_julho', 'trilha_bau_cachoeira_das_andorinhas', 'avenida_das_andorinhas_386207748', 'way_115208211', 'way_836343202', 'way_115208253', 'way_196416342', '

In [5]:
from src.helpers.graph_helper import display_graph

START, GOAL = 'rua_rio_piracicaba', 'rua_rio_piracicaba'
# Gera grafo interativo (Dash Cytoscape) e exibe no notebook
display_graph(G, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500, start=START, goal=GOAL)

## 2. Cenário baseline — comparação dos três algoritmos

In [17]:
reset_scenarios(G)

lat_d, (path_d, cost_d) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a, (path_a, cost_a) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl, (path_dl, cost_dl) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)
display_graph(G, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500, start=START, goal=GOAL)

print('Dijkstra:  caminho =', path_d, '| custo =', round(cost_d, 2))
print('A*:       caminho =', path_a, '| custo =', round(cost_a, 2))
print('D* Lite:  caminho =', path_dl, '| custo =', round(cost_dl, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d:.4f} ms | A*: {lat_a:.4f} ms | D* Lite: {lat_dl:.4f} ms')

Dijkstra:  caminho = ['rua_rio_piracicaba'] | custo = 0.0
A*:       caminho = ['rua_rio_piracicaba'] | custo = 0.0
D* Lite:  caminho = ['rua_rio_piracicaba'] | custo = 0.0
  Latência média (200 exec.): Dijkstra: 0.0012 ms | A*: 0.0030 ms | D* Lite: 0.0027 ms


## 3. Casos de teste com impeditivos no caminho (Ouro Preto)

Mantendo **START=rio_piracicaba** e **GOAL=americo_rene_gianetti**. Cada caso aplica interdições, congestionamento ou chuva em ruas do caminho baseline (rio_piracicaba → joao_de_paiva → hugo_soderi → americo_rene_gianetti) para forçar desvio ou custo maior.

In [16]:
def run_case(name, setup_fn):
    """Aplica cenário, roda os 3 algoritmos e retorna (path_d, cost_d, path_a, cost_a, path_dl, cost_dl)."""
    reset_scenarios(G)
    setup_fn(G)
    l_d, (path_d, cost_d) = measure_latency_ms(lambda: dijkstra(G, START, GOAL))
    l_a, (path_a, cost_a) = measure_latency_ms(lambda: a_star(G, START, GOAL))
    l_dl, (path_dl, cost_dl) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL))
    print(f"  {name}")
    print(f"    Dijkstra:  {path_d} | latencia = {l_d:.4f} ms | custo = {round(cost_d, 2)}")
    print(f"    A*:        {path_a} | latencia = {l_a:.4f} ms | custo = {round(cost_a, 2)}")
    print(f"    D* Lite:   {path_dl} | latencia = {l_dl:.4f} ms | custo = {round(cost_dl, 2)}")
    return path_d, cost_d, path_a, cost_a, path_dl, cost_dl

In [ ]:
# Caso 1: interdição rio_piracicaba ↔ joao_de_paiva
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 1 — interdição (rio_piracicaba ↔ joao_de_paiva)",
    lambda g: apply_barriers(g, OURO_PRETO_BLOCKED_CASE1),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso1.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

In [ ]:
# Caso 2: congestionamento em trechos do caminho
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 2 — congestionamento (joao_de_paiva↔hugo_soderi, hugo_soderi↔americo_rene_gianetti)",
    lambda g: apply_congestion_scenario_by_edge(g, OURO_PRETO_CONGESTED_CASE2),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso2.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

In [ ]:
# Caso 3: chuva em regiões (joao_de_paiva, hugo_soderi, americo_rene_gianetti)
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 3 — chuva em regiões do caminho",
    lambda g: apply_climate_scenario_by_region(g, OURO_PRETO_RAIN_REGIONS_CASE3),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso3.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

In [ ]:
# Caso 4: interdição + congestionamento
def setup_case4(g):
    apply_barriers(g, OURO_PRETO_BLOCKED_CASE4)
    apply_congestion_scenario_by_edge(g, OURO_PRETO_CONGESTED_CASE4)
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case("Caso 4 — interdição + congestionamento", setup_case4)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso4.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

In [ ]:
# Caso 5: interdição joao_de_paiva ↔ hugo_soderi
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 5 — interdição (joao_de_paiva ↔ hugo_soderi)",
    lambda g: apply_barriers(g, OURO_PRETO_BLOCKED_CASE5),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso5.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

In [ ]:
# Caso 6: congestionamento em todas as arestas do caminho baseline
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 6 — congestionamento em todo o caminho baseline",
    lambda g: apply_congestion_scenario_by_edge(g, OURO_PRETO_CONGESTED_CASE6),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso6.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

In [ ]:
# Caso 7: interdição hugo_soderi↔americo_rene_gianetti + chuva em regiões
def setup_case7(g):
    apply_barriers(g, OURO_PRETO_BLOCKED_CASE7)
    apply_climate_scenario_by_region(g, OURO_PRETO_RAIN_REGIONS_CASE7)
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case("Caso 7 — interdição + chuva em regiões", setup_case7)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso7.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

## 3. Cenário de evento — ruas interditadas (Diogo de Vasconcelos)

O sistema deve sugerir rotas alternativas (Rua do Pilar ou Xavier da Veiga).

In [ ]:
apply_event_scenario(G)

lat_d_e, (path_d_e, cost_d_e) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_e, (path_a_e, cost_a_e) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_e, (path_dl_e, cost_dl_e) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('Com ruas interditadas:')
print('Dijkstra:  caminho =', path_d_e, '| custo =', round(cost_d_e, 2))
print('A*:       caminho =', path_a_e, '| custo =', round(cost_a_e, 2))
print('D* Lite:  caminho =', path_dl_e, '| custo =', round(cost_dl_e, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_e:.4f} ms | A*: {lat_a_e:.4f} ms | D* Lite: {lat_dl_e:.4f} ms')

## 4. Cenário climático — chuva (ladeiras íngremes +200%)

In [ ]:
reset_scenarios(G)
apply_climate_scenario(G, rain_multiplier=2.0)

lat_d_c, (path_d_c, cost_d_c) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_c, (path_a_c, cost_a_c) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_c, (path_dl_c, cost_dl_c) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('Com chuva (ladeiras +200%):')
print('Dijkstra:  caminho =', path_d_c, '| custo =', round(cost_d_c, 2))
print('A*:       caminho =', path_a_c, '| custo =', round(cost_a_c, 2))
print('D* Lite:  caminho =', path_dl_c, '| custo =', round(cost_dl_c, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_c:.4f} ms | A*: {lat_a_c:.4f} ms | D* Lite: {lat_dl_c:.4f} ms')

### 4.1 Cenários combinados (Ouro Preto)

Bloqueio + chuva; bloqueio + chuva + alta densidade; chuva e congestionamento **por região** (alguns bairros com chuva/trânsito, outros não).

In [ ]:
# Cenário A: bloqueio (Diogo de Vasconcelos) + chuva
reset_scenarios(G)
apply_event_scenario(G)
apply_climate_scenario(G, rain_multiplier=2.0)

lat_d_ac, (path_d_ac, cost_d_ac) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_ac, (path_a_ac, cost_a_ac) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_ac, (path_dl_ac, cost_dl_ac) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)
display_graph(G, start=START, goal=GOAL, path=path_d_ac, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

print('A — Bloqueio + Chuva:')
print('  Dijkstra:', path_d_ac, '| custo =', round(cost_d_ac, 2))
print('  A*:     ', path_a_ac, '| custo =', round(cost_a_ac, 2))
print('  D* Lite:', path_dl_ac, '| custo =', round(cost_dl_ac, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_ac:.4f} ms | A*: {lat_a_ac:.4f} ms | D* Lite: {lat_dl_ac:.4f} ms')

In [ ]:
reset_scenarios(G)
apply_event_scenario(G)
apply_climate_scenario(G, rain_multiplier=2.0)
apply_congestion_scenario_by_edge(G, {("lagoa", "morro_santana"): 2.0})

lat_d_b, (path_d_b, cost_d_b) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_b, (path_a_b, cost_a_b) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_b, (path_dl_b, cost_dl_b) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)
display_graph(G, path=path_d_b, start=START, goal=GOAL, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)


print('B — Bloqueio + Chuva + Congestionamento:')
print('  Dijkstra:', path_d_b, '| custo =', round(cost_d_b, 2))
print('  A*:     ', path_a_b, '| custo =', round(cost_a_b, 2))
print('  D* Lite:', path_dl_b, '| custo =', round(cost_dl_b, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_b:.4f} ms | A*: {lat_a_b:.4f} ms | D* Lite: {lat_dl_b:.4f} ms')

In [ ]:
# Cenário C: chuva por região + congestionamento por via (vias centro↔são_jose congestionadas)
reset_scenarios(G)
apply_climate_scenario_by_region(G, {"centro":9.0, "praca_tiradentes": 9.0, "campus": 1.0})
apply_congestion_scenario_by_edge(G, {("centro", "sao_jose"): 5.0, ("sao_jose", "centro"): 5.0})
display_graph(G, start=START, goal=GOAL, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

lat_d_c2, (path_d_c2, cost_d_c2) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_c2, (path_a_c2, cost_a_c2) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_c2, (path_dl_c2, cost_dl_c2) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('C — Chuva por região + Congestionamento por via (centro↔são_jose congestionados):')
print('  Dijkstra:', path_d_c2, '| custo =', round(cost_d_c2, 2))
print('  A*:     ', path_a_c2, '| custo =', round(cost_a_c2, 2))
print('  D* Lite:', path_dl_c2, '| custo =', round(cost_dl_c2, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_c2:.4f} ms | A*: {lat_a_c2:.4f} ms | D* Lite: {lat_dl_c2:.4f} ms')

## 5. Latência de re-roteamento (< 100 ms para tempo real)

In [ ]:
reset_scenarios(G)
repetitions = 500

lat_d, (path_d, cost_d) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), repetitions)
lat_a, (path_a, cost_a) = measure_latency_ms(lambda: a_star(G, START, GOAL), repetitions)
lat_dl, (path_dl, cost_dl) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), repetitions)

print(f'Latência média ({repetitions} execuções):')
print(f'  Dijkstra: {lat_d:.4f} ms')
print(f'  A*:       {lat_a:.4f} ms')
print(f'  D* Lite:  {lat_dl:.4f} ms')
print('  Meta: < 100 ms para uso em tempo real')

## 6. Tabela resumo para o relatório

In [ ]:
import pandas as pd

reset_scenarios(G)
rows = []
for name, fn in [('Dijkstra', dijkstra), ('A*', a_star), ('D* Lite', d_star_lite)]:
    lat, (path, cost) = measure_latency_ms(lambda: fn(G, START, GOAL), 200)
    rows.append({'Algoritmo': name, 'Custo': round(cost, 2), 'Latência (ms)': round(lat, 4)})

df = pd.DataFrame(rows)
df